In [ ]:
import osmnx as ox
import matplotlib.pyplot as plt
import os
import matplotlib as mpl
import networkx as nx

# Python import malarky
import sys
import os

parent_dir = os.path.abspath("..")

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import zone_coords

In [ ]:
def fetch_road_graph():
    boroughs = ['Bronx, New York, USA', 'Brooklyn, New York, USA',
                'Manhattan, New York, USA', 'Queens, New York, USA',
                'Staten Island, New York, USA']
    
    print("Getting graph of NYC boroughs")
    
    G = ox.graph_from_place(boroughs, network_type='drive', simplify=True)
    
    G = ox.add_edge_speeds(G)
    G = ox.add_edge_travel_times(G)

    return G

In [ ]:
def display_real_city(G):
    fig, ax = plt.subplots(figsize=(15, 15))

    speeds = nx.get_edge_attributes(G, 'speed_kph')

    # Normalize
    min_speed = min(speeds.values())
    max_speed = max(speeds.values())
    norm = mpl.colors.Normalize(vmin=min_speed, vmax=max_speed)
    
    colormap = plt.cm.viridis
    edge_colors = [colormap(norm(speeds.get(edge, min_speed))) for edge in G.edges]

    # Plot the graph with edge colors based on speed
    ox.plot_graph(G, ax=ax, node_size=0, edge_linewidth=0.2, edge_color=edge_colors, show=False)
    
    sm = plt.cm.ScalarMappable(cmap=colormap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label("Speed (kph)")
    
    plt.title("NYC Road Network with Edge Speeds")
    plt.tight_layout()
    plt.savefig("nyc_roads_map_with_speeds.png", dpi=450)
    print("Map saved as 'nyc_roads_map_with_speeds.png'")
    plt.show()

In [ ]:
def get_zones():
    zone_lookup = zone_coords.get_coords_as_dict()

get_zones()

In [ ]:
# Returns zones, street_graph, trips, width, height
def generate_real_city_data(force=False):
    # Don't fetch the graph if it already exists
    if not os.path.exists("nyc_roads.graphml") or force:
        G = fetch_road_graph()

        ox.save_graphml(G, filepath="nyc_roads.graphml")
        print("Graph saved as 'nyc_roads.graphml'")
    else:
        G = ox.load_graphml("nyc_roads.graphml")
        print("Graph loaded from 'nyc_roads.graphml'")
    
    zones = get_zones()
    street_graph = G
    trips = get_trips()
    width, height = get_dimensions()
    
    return zones, street_graph, trips, width, height